### Auto Encoder Developement

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import keras_tuner as kt
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold,GridSearchCV,RandomizedSearchCV,train_test_split,StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score,r2_score,roc_auc_score,precision_score,recall_score,f1_score,mean_squared_error
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=True)

In [2]:
col_names = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 'T2', 'T24'
             , 'T30', 'T50','P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30'
             , 'phi', 'NRf', 'NRc', 'BPR','farB', 'htBleed', 'Nf_dmd',
             'PCNfR_dmd','W31', 'W32', 's22', 's23']

Import the training, test, and remaining useful life data and remove all NaN values.

In [3]:
train = pd.read_csv("train.txt", sep=' ', names=col_names)
test = pd.read_csv("test.txt", sep=' ', names=col_names)
RUL = pd.read_csv("RUL.txt", sep=' ', names=['remaining_cycles', 'Nan'])
RUL.head()

,remaining_cycles,Nan
0,112,NaN
1,98,NaN
2,69,NaN
3,82,NaN
4,91,NaN


In [4]:
train.drop(columns=['s22', 's23'], axis=1, inplace=True)
test.drop(columns=['s22', 's23'], axis=1, inplace=True)
RUL.drop(columns=['Nan'], axis=1, inplace=True)

In [5]:
RUL['id'] = RUL.index + 1

RUL.head()

,remaining_cycles,id
0,112,1
1,98,2
2,69,3
3,82,4
4,91,5


The dataset is made up of sensor data for 100 different engines, as can be seen below.

In [6]:
print('Engine IDs:', train.id.unique())

Engine IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


## Processing Data

A testing dataset is provided where the engines are not ran until failure, and a complimentary vector is provided for the remaining useful life (RUL) for each engine ID. In order to match the engines in the test set with their corresponding RUL, we must concatenate the two datasets.

Since the engines in the test set are ran until failure, you can calculate the remaining cycles until failure by subtracting the current cycle from the maximum cycle for each engine ID. He does this for the test set as well. Since the remaining useful life for all the test engines are given, as well as their current cycle, the maximum cycles for each engine can be calculated.

In [7]:
train['remaining_cycles'] = train.groupby(['id'])['cycle'].transform(max)-train['cycle']

In [8]:
# Maximum cycles = cycles at test end + remaining useful life

maxCycles = pd.DataFrame(test.groupby('id')['cycle'].max()).reset_index()
maxCycles.columns = ['id', 'max_tested']
maxCycles['max_cycles'] = RUL['remaining_cycles'] + maxCycles['max_tested']
maxCycles.drop(['max_tested'], axis=1, inplace=True)

In [9]:
maxCycles.head()

,id,max_cycles
0,1,143
1,2,147
2,3,195
3,4,188
4,5,189


In [10]:
test = test.merge(maxCycles, on=['id'], how='left')

In [11]:
test['remaining_cycles'] = test['max_cycles'] - test['cycle']
test.drop(['max_cycles'], axis=1, inplace=True)

In [12]:
Xtrain = train.drop(['remaining_cycles'], axis=1)
y = train.remaining_cycles

In [13]:
meanTrain = Xtrain.loc[:, Xtrain.columns != 'cycle'].rolling(10).mean()
meanTrain = meanTrain.join(Xtrain['cycle'])
X = meanTrain[Xtrain.columns]
X.dropna(inplace=True)
y = y[9:]

C:\Users\Arash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [14]:
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
gen = MinMaxScaler(feature_range=(0,1))
X = gen.fit_transform(X)
X = pd.DataFrame(X)
X = np.nan_to_num(X)

pt = PowerTransformer()
X = pt.fit_transform(X)
X = np.array(X)

C:\Users\Arash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_data.py:3251: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [15]:
AR = 10

def create_dataset(X, AR=AR):
    data = []
    for i in range(len(X)-AR-1):
        data.append(X[i:(i+AR)])
    return np.array(data)

In [16]:
X = create_dataset(X, AR)
y_train = y[AR+1:]

In [17]:
X.shape

(20611, 10, 26)

In [18]:
class AutoEncoderLSTM(Model):
    def __init__(self, latent_dim, input_dim):
        super(AutoEncoderLSTM, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential()
        
        # Encoder
        self.encoder.add(layers.LSTM(input_dim[1], activation="relu", input_shape=(input_dim[1], input_dim[2]), return_sequences=True))
        self.encoder.add(layers.LSTM(int(input_dim[1]/2), activation="relu", return_sequences=True))
        self.encoder.add(layers.LSTM(latent_dim, activation="relu"))

        # Decoder
        self.decoder = tf.keras.Sequential()
        self.decoder.add(layers.RepeatVector(input_dim[1]))
        self.decoder.add(layers.LSTM(input_dim[1], activation="relu", input_shape=(input_dim[1], input_dim[2]), return_sequences=True))
        self.decoder.add(layers.TimeDistributed(layers.Dense(input_dim[2])))
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [19]:
def make_lstm_model(autoencoder, loss_type, activation_type, input):
    autoencoder.encoder.trainable = False
    model = keras.Sequential()
    model.add(autoencoder.encoder)
    model.add(layers.RepeatVector(input[2]))
    model.add(layers.LSTM(128, activation=activation_type, return_sequences=True))
    model.add(layers.LSTM(64, activation=activation_type, return_sequences=False))
    model.add(layers.TimeDistributed(layers.Dense(16)))
    model.add(layers.TimeDistributed(layers.Dense(1)))
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model

In [20]:
def plot_loss(fit):
    plt.figure(figsize=(16,9))
    plt.plot(fit.history['loss'], label='loss')
    plt.plot(fit.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Remaining Cycles Error')
    plt.legend(loc=0, prop={'size':26})
    plt.grid(True)
    plt.show()

In [21]:
X.shape

(20611, 10, 26)

In [22]:
latent_dim = 9
tf.keras.backend.clear_session()
autoencoder = AutoEncoderLSTM(latent_dim,X.shape)
autoencoder.compile(optimizer="adam", loss="mean_squared_error")
autoencoder.build(X.shape)
autoencoder.summary()

Model: "auto_encoder_lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 9)                 2340      
_________________________________________________________________
sequential_1 (Sequential)    (20611, 10, 26)           1086      
Total params: 3,426
Trainable params: 3,426
Non-trainable params: 0
_________________________________________________________________


In [23]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10)
autoencoder.fit(X, X, epochs=1000, shuffle=True, validation_data=(X, X), verbose=True, callbacks=callback)
autoencoder.encoder.trainable = False
autoencoder.encoder.trainable

Epoch 1/1000
645/645 [==============================] - 9s 9ms/step - loss: 0.4218 - val_loss: 0.3002
Epoch 2/1000
645/645 [==============================] - 6s 9ms/step - loss: 0.2847 - val_loss: 0.2710
Epoch 3/1000
645/645 [==============================] - 6s 9ms/step - loss: 0.2598 - val_loss: 0.2486
Epoch 4/1000
645/645 [==============================] - 7s 11ms/step - loss: 0.2387 - val_loss: 0.2318
Epoch 5/1000
645/645 [==============================] - 6s 10ms/step - loss: 0.2252 - val_loss: 0.2195
Epoch 6/1000
645/645 [==============================] - 6s 10ms/step - loss: 0.2125 - val_loss: 0.2090
Epoch 7/1000
645/645 [==============================] - 7s 11ms/step - loss: 0.2064 - val_loss: 0.2060
Epoch 8/1000
645/645 [==============================] - 7s 11ms/step - loss: 0.2041 - val_loss: 0.2034
Epoch 9/1000
645/645 [==============================] - 6s 9ms/step - loss: 0.2028 - val_loss: 0.2037
Epoch 10/1000
645/645 [==============================] - 5s 8ms/step - loss: 

False

In [24]:
autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 10)            1480      
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 5)             320       
_________________________________________________________________
lstm_2 (LSTM)                (None, 9)                 540       
Total params: 2,340
Trainable params: 0
Non-trainable params: 2,340
_________________________________________________________________


In [25]:
lstmModel = make_lstm_model(autoencoder, "mean_squared_error", "sigmoid", [X.shape[2], X.shape[1]])
lstmModel.build([X.shape[2], X.shape[1]])
lstmModel.summary()

IndexError: list index out of range

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
history = lstmModel.fit(X, y, batch_size=512, epochs=500, validation_split=0.15, callbacks=callback, verbose=1)

In [ ]:
plot_loss(history)